In [17]:
from pathlib import Path
from typing import Tuple, Dict, Callable
import cProfile
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import plotly.express as pexp
from main import load_prices, log_returns, optimise_cvar, optimize_mean_variance, simulate_top_n

In [18]:
DATA_PATH = Path("stocks_clean_aligned_v2.csv")
# BETA = 0.8
TARGET_RETURN = 1e-5
SHORT_CAP = 0
FREQ = "M"  # monthly
INITIAL_CASH = 1000.0
SHOW_ABS_GT = 1e-10
BASELINES = ["MSFT", "MRK", "GOOGL", "CVX"]
window = 5

In [19]:

def walk_forward(
    prices: pd.DataFrame,
    optimiser: Callable[..., dict],
    init_cash: float = 100.0,
    freq: str = "M",
    label: str = "strategy",
) -> Tuple[pd.Series, pd.DataFrame]:
    """Generic walk‑forward wrapper for any single‑period optimiser."""
    periods = pd.period_range(prices.index.min(), prices.index.max(), freq=freq)

    eq_curve, w_log = [], []
    equity = init_cash

    for i in range(len(periods) - 1):
        # optimise with all info ≤ period end
        R = log_returns(prices.loc[: periods[i].end_time])
        if R.empty:
            continue

        res = optimiser(R)
        w = res["weights"]

        # hold during next period
        hold_px = prices.loc[periods[i].end_time : periods[i + 1].end_time]
        if len(hold_px) < 2:
            continue
        ret = np.dot((hold_px.iloc[-1] / hold_px.iloc[0] - 1), w.fillna(0.0))
        equity *= 1 + ret

        eq_curve.append((periods[i + 1].end_time, equity))
        w_log.append((periods[i + 1].end_time, w))

    equity_ser = pd.Series(
        [v for _, v in eq_curve],
        index=pd.DatetimeIndex([d for d, _ in eq_curve], name="date"),
        name=label,
    )
    weights_df = (
        pd.concat([w.rename(d) for d, w in w_log], axis=1)
        .T.sort_index()
        .rename_axis("date")
    )
    return equity_ser, weights_df


def pct_ret(series: pd.Series) -> pd.Series:
    return series.pct_change().dropna()


def rolling_cvar(ret: pd.Series, beta: float, window: int) -> pd.Series:
    def cvar(x):
        losses = x[x < 0]
        if losses.empty:
            return 0.0
        var = np.quantile(losses, 1 - beta)
        return losses[losses <= var].mean()

    return ret.rolling(window).apply(cvar, raw=False)


def drawdown_curve(equity: pd.Series) -> pd.Series:
    cummax = equity.cummax()
    return (equity - cummax) / cummax



def line_chart(series_dict: Dict[str, pd.Series], title: str, y_title: str):
    fig = go.Figure()
    for name, s in series_dict.items():
        s.fillna(0, inplace=True)
        fig.add_trace(go.Scatter(x=s.index, y=s.values, mode="lines", name=name))
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title=y_title,
        legend_title="Series",
        template="plotly_white",
    )
    fig.show()


def boxplot(returns: Dict[str, pd.Series], title: str):
    fig = go.Figure()
    for name, r in returns.items():
        fig.add_trace(
            go.Box(
                x=r.values,
                name=name,
                boxmean="sd",
                orientation="h",
            )
        )
    fig.update_layout(title=title, template="plotly_white", xaxis_title="Return")
    fig.show()




In [20]:
prices = load_prices(DATA_PATH)


In [ ]:
# equity_cvar_90, w_cvar_90 = walk_forward(
#     prices,
#     lambda R: optimise_cvar(R, 0.9, TARGET_RETURN, SHORT_CAP),
#     INITIAL_CASH,
#     FREQ,
#     label="CVaR 90",
# )



In [ ]:


with cProfile.Profile() as pr:
    equity_cvar_90, w_cvar_90 = walk_forward(
        prices,
        lambda R: optimise_cvar(R, 0.9, TARGET_RETURN, SHORT_CAP),
        INITIAL_CASH,
        FREQ,
        label="CVaR 90",
    )

pr.dump_stats("walk_forward_profile_cvar_90.prof")


In [23]:



with cProfile.Profile() as pr:
    equity_cvar_95, w_cvar_95 = walk_forward(
        prices,
        lambda R: optimise_cvar(R, 0.95, TARGET_RETURN, SHORT_CAP),
        INITIAL_CASH,
        FREQ,
        label="CVaR 95",
    )

pr.dump_stats("walk_forward_profile_cvar_95.prof")

In [ ]:


with cProfile.Profile() as pr:
    equity_cvar_80, w_cvar_80 = walk_forward(
        prices,
        lambda R: optimise_cvar(R, 0.8, TARGET_RETURN, SHORT_CAP),
        INITIAL_CASH,
        FREQ,
        label="CVaR 80",
    )

pr.dump_stats("walk_forward_profile_cvar_80.prof")

In [ ]:


with cProfile.Profile() as pr:
    equity_cvar_99, w_cvar_99 = walk_forward(
        prices,
        lambda R: optimise_cvar(R, 0.99, TARGET_RETURN, SHORT_CAP),
        INITIAL_CASH,
        FREQ,
        label="CVaR 99",
    )

pr.dump_stats("walk_forward_profile_cvar_99.prof")

In [25]:



with cProfile.Profile() as pr:
    equity_mv, w_mv = walk_forward(
        prices,
        lambda R: optimize_mean_variance(R, TARGET_RETURN, SHORT_CAP),
        INITIAL_CASH,
        FREQ,
        label="MVar",
    )

pr.dump_stats("walk_forward_profile_mvar.prof")



In [26]:
top3_eq = simulate_top_n(prices.resample(FREQ).last(), 3, INITIAL_CASH)

C:\Users\uzman\AppData\Local\Temp\ipykernel_44224\3533111393.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  top3_eq = simulate_top_n(prices.resample(FREQ).last(), 3, INITIAL_CASH)


In [27]:
baselines: Dict[str, pd.Series] = {}
for tkr in BASELINES:
    if tkr not in prices.columns:
        raise KeyError(f"Ticker '{tkr}' not found.")
    px = prices[tkr].sort_index().reindex(equity_cvar_80.index, method="ffill")
    baselines[tkr] = INITIAL_CASH * px / px.iloc[0]
    baselines[tkr].name = tkr

In [ ]:
rows = ["CVaR_80", "CVaR_95", "CVaR_90","CVaR_99", "MVar", "TOP3"] + list(baselines.keys())
finals = [
    equity_cvar_80.iloc[-1],
    equity_cvar_95.iloc[-1],
    equity_cvar_90.iloc[-1],
    w_cvar_99.iloc[-1],
    equity_mv.iloc[-1],
    top3_eq.iloc[-1],
] + [s.iloc[-1] for s in baselines.values()]
cumret = [(v / INITIAL_CASH - 1) * 100 for v in finals]
summary = (
    pd.DataFrame({"final_$": finals, "cum_%": cumret}, index=rows)
    .round(2)
    .sort_values(by="cum_%")
    .reset_index()
)

In [29]:
summary

,index,final_$,cum_%
0,CVX,1129.64,12.96
1,MRK,1553.57,55.36
2,TOP3,1802.77,80.28
3,GOOGL,2380.73,138.07
4,MSFT,3596.35,259.64
5,CVaR_95,7536.64,653.66
6,CVaR_90,7571.41,657.14
7,MVar,7818.84,681.88
8,CVaR_80,9069.32,806.93


In [30]:

df = summary.sort_values(by="cum_%").copy()
df["cum_%_str"] = df["cum_%"].map(lambda x: f"{x:.1f}%")

fig = pexp.bar(
    df,
    x="index",
    y="cum_%",
    text="cum_%_str",
    color="cum_%",  # Color intensity by percentage
    color_continuous_scale="Blues",  # Choose your color theme
    title="Cumulative Percentage by Strategy"
)

# Customize layout
fig.update_layout(
    xaxis_title="Strategy",
    yaxis_title="Cumulative %",
    title_x=0.5,
    plot_bgcolor="white",
    font=dict(size=15),
    xaxis=dict(showgrid=False),
    yaxis=dict(ticksuffix="%", showgrid=True, gridcolor="lightgray"),
)

# Set text position and size
fig.update_traces(
    textposition="inside",
    textfont_size=12
)

fig.show()


In [ ]:
# print last weights for both strategies
print("\n===== LAST CVaR WEIGHTS (|w| ≥ {:.4f}) =====".format(SHOW_ABS_GT))
print(w_cvar_80.iloc[-1][w_cvar_80.iloc[-1].abs() >= SHOW_ABS_GT].round(6).to_string())

print("\n===== LAST MVar WEIGHTS (|w| ≥ {:.4f}) =====".format(SHOW_ABS_GT))
print(w_mv.iloc[-1][w_mv.iloc[-1].abs() >= SHOW_ABS_GT].round(6).to_string())

# ── equity curves ──────────────────────────────────────────────────── #
series_equity = {
    "CVaR_95": equity_cvar_95,
    "CVaR_90": equity_cvar_90,
    "CVaR_80": equity_cvar_80,
    "CVaR_99": equity_cvar_99,
    "MVar": equity_mv,
    "TOP3": top3_eq,
} | baselines
line_chart(series_equity, f"Equity curve — re-balance = {FREQ}", "Portfolio value ($)")

# ── KPI charts (rolling CVaR, σ, Sharpe, drawdowns, boxplot) ───────── #
returns = {k: pct_ret(v) for k, v in series_equity.items()}


# 1. Rolling CVaR
# rolling_cvar_series = {k: rolling_cvar(r, BETA, window) for k, r in returns.items()}
# line_chart(
#     rolling_cvar_series,
#     f"Rolling {int(BETA*100)}% CVaR (window={window})",
#     "CVaR (mean loss)",
# )

# 2. Rolling volatility σ
rolling_vol = {k: r.rolling(window).std() for k, r in returns.items()}
line_chart(rolling_vol, f"Rolling volatility σ (window={window})", "Std‑dev of returns")

# 3. Rolling Sharpe
rolling_sharpe = {
    k: r.rolling(window).mean() / r.rolling(window).std() for k, r in returns.items()
}
line_chart(rolling_sharpe, f"Rolling Sharpe ratio (window={window})", "Sharpe ≈ μ/σ")

# 4. Drawdowns
drawdowns = {k: drawdown_curve(eq) for k, eq in series_equity.items()}
line_chart(drawdowns, "Drawdown curves", "Drawdown (%)")

# 5. Box‑plot of period returns
boxplot(returns, "Distribution of period returns")


# if __name__ == "__main__":
#     _run_cli()


===== LAST CVaR WEIGHTS (|w| ≥ 0.0000) =====
company
CVX      0.016653
GOOGL    0.165433
JNJ      0.201692
MRK      0.190432
PFE      0.072406
XOM      0.353385

===== LAST MVar WEIGHTS (|w| ≥ 0.0000) =====
company
COP      0.017186
GOOGL    0.198778
JNJ      0.166940
MRK      0.155129
PFE      0.143891
XOM      0.318076


In [32]:
from walk_forward_explain import walk_forward_explain


In [33]:
equity_cvar_90, w_cvar_90, rc_cvar_90 = walk_forward_explain(
    prices,
    lambda R: optimise_cvar(R, 0.90, TARGET_RETURN, SHORT_CAP),
    INITIAL_CASH,
    FREQ,
    label="CVaR 90",
    rf=1,
    as_pct=True
)
equity_cvar_80, w_cvar_80, rc_cvar_80 = walk_forward_explain(
    prices,
    lambda R: optimise_cvar(R, 0.80, TARGET_RETURN, SHORT_CAP),
    INITIAL_CASH,
    FREQ,
    label="CVaR 80",
    rf=1,
    as_pct=False,
)




In [34]:
equity_mv, w_mv, rc_mv = walk_forward_explain(
    prices,
    lambda R: optimize_mean_variance(R, TARGET_RETURN, SHORT_CAP),
    INITIAL_CASH,
    FREQ,
    label="Mean-Variance",
    rf=1,
    as_pct=False,
)

In [35]:
# from explainability import plot_risk_contributions

# fig = plot_risk_contributions(
#     {
#         "CVaR 90": rc_cvar_90.mean(),
#         "CVaR 80": rc_cvar_80.mean(),
#         "Mean-Var": rc_mv.mean(),
#         # add more strategies here e.g. {"CVaR 95": rc_cvar_95.mean()}
#     },
#     chart="pie",                            # or "pie"
#     title="Average risk attribution (2010-2015 back-test)",
# )
# # fig.write_html("risk_attribution.html")     # drop into PowerPoint via Web Viewer
# fig.show()

In [36]:

top_per_period = pd.DataFrame({
    "date":   rc_cvar_80.index,
    "asset":  rc_cvar_80.idxmax(axis=1),    # ticker with largest share
    "share":  rc_cvar_80.max(axis=1),       # that share in %
})
top_per_period["share"] = top_per_period["share"].round(2)  # nice labels

# 2)  Plot – one bar per period, coloured by the winning asset
fig_top = pexp.bar(
    top_per_period,
    x="date",
    y="share",
    color="asset",
    text="share",
    title="Who dominates portfolio risk each month?",
    labels={"share": "Sharpe contribution"},
    height=400,
)
# fig_top.update_traces(texttemplate="%{text:.2f}%", textposition="inside")
fig_top.update_layout(
    yaxis=dict(range=[0, top_per_period["share"].max() * 1.15]),
    xaxis_title="Rebalance date",
    showlegend=True,
)
fig_top.show()                 # or .write_html("top_contributor.html")


In [37]:

top_per_period = pd.DataFrame({
    "date":   rc_mv.index,
    "asset":  rc_mv.idxmax(axis=1),    # ticker with largest share
    "share":  rc_mv.max(axis=1),       # that share in %
})
top_per_period["share"] = top_per_period["share"].round(2)  # nice labels

# 2)  Plot – one bar per period, coloured by the winning asset
fig_top = pexp.bar(
    top_per_period,
    x="date",
    y="share",
    color="asset",
    text="share",
    title="Who dominates portfolio risk each month?",
    labels={"share": "Sharpe contribution"},
    height=400,
)
# fig_top.update_traces(texttemplate="%{text:.2f}%", textposition="inside")
fig_top.update_layout(
    yaxis=dict(range=[0, top_per_period["share"].max() * 1.15]),
    xaxis_title="Rebalance date",
    showlegend=True,
)

fig_top.show()                 # or .write_html("top_contributor.html")


In [74]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ── 1.  pick the single-period contribution vector ────────────────────────────
# rc_cvar_80.iloc[30] is assumed to be a Pandas Series: index = asset names,
# values = marginal Sharpe contributions
sharpe_contrib = rc_cvar_80.iloc[6]

# ── 2.  reorder for visual impact  (largest |Δ| on top) ───────────────────────
sharpe_contrib = sharpe_contrib.reindex(
    sharpe_contrib.abs().sort_values(ascending=False).index
)

# ── 3.  build bar colours & hover text ────────────────────────────────────────
colors = np.where(sharpe_contrib >= 0, "rgba(34,161,57,0.8)",  # greenish
                                   "rgba(218,39,39,0.8)")      # reddish
hover = (sharpe_contrib.index + "<br>Δ-Sharpe: " +
         sharpe_contrib.map("{:+.4f}".format))

# ── 4.  create the figure ─────────────────────────────────────────────────────
fig = go.Figure(go.Bar(
    x=sharpe_contrib.values,
    y=sharpe_contrib.index,
    orientation="h",
    marker_color=colors,
    text=sharpe_contrib.map("{:+.4f}".format),
    textposition="outside",
    hovertext=hover,
    hoverinfo="text",
))

fig.update_layout(
    title=dict(
        text="Marginal Sharpe Contributions<br>"
             f"<span style='font-size:0.8em'>(sum = {sharpe_contrib.sum():+.4e})</span>",
        y=0.95
    ),
    xaxis_title="Δ Sharpe",
    yaxis_title="Asset",
    yaxis=dict(autorange="reversed"),  # largest at top
    bargap=0.25,
    template="plotly_white",
    height=max(400, 20 * len(sharpe_contrib))  # auto-stretch for many assets
)

fig.show()


In [71]:
ka = rc_cvar_80.iloc[30]
# k = ka['weights']
k = ka[ka>1e-10]
# k , k.sum(), 
# p = k
pexp.pie(k,names=k.index, values=k.values)
w_cvar_80.iloc[30]

company
AAPL     9.475766e-03
COP      3.037150e-02
CVX      2.723999e-14
GOOGL    1.874350e-01
JNJ      6.936272e-02
MRK      1.919284e-01
MSFT     2.839389e-14
PFE      1.820390e-01
XOM      3.293877e-01
Name: 2017-08-31 23:59:59.999999999, dtype: float64

In [38]:
# rc_mv.abs()

In [39]:
# rc_mv.max(axis=1)

In [40]:
import pandas as pd

In [41]:
periods = pd.period_range(prices.index.min(), prices.index.max(), freq=FREQ)
p = 2
R = log_returns(prices.loc[: periods[p].end_time])

In [42]:


ka = optimise_cvar(R, 0.80, TARGET_RETURN, SHORT_CAP)
k = ka['weights']
k = k[k>1e-3]

k , k.sum()

(company
 COP      0.045597
 GOOGL    0.097989
 JNJ      0.548025
 MRK      0.138019
 PFE      0.170370
 Name: weight, dtype: float64,
 np.float64(0.9999999999900633))

In [43]:
ka

{'weights': company
 AAPL     4.771334e-12
 COP      4.559669e-02
 CVX      9.777097e-13
 GOOGL    9.798934e-02
 JNJ      5.480251e-01
 MRK      1.380186e-01
 MSFT     3.092940e-13
 PFE      1.703703e-01
 XOM      3.878290e-12
 Name: weight, dtype: float64,
 'expected_return': 0.00020522627493911922,
 'VaR': array(0.0129318),
 'CVaR': np.float64(0.030449983354779463),
 'status': 'optimal'}